In [1]:
from maad import sound, util, rois
import matplotlib.pyplot as plt

import numpy as np
from scipy.io import wavfile
from scipy.signal import resample_poly, find_peaks
import os

In [38]:
#this only really matters to be able to see the spectrogram
dB_adj = 70

target_path = "lf_target.wav"
d_template = "d_template.wav"
b_template = "b_template_3.wav"

template_dict = {
    d_template: .22,
    b_template: .12
}

call_length = {
    d_template: 8,
    b_template: 25
}

def create_template_spectrogram(template_path, adjustment_factor):
    s, fs = sound.load(template_path)
    Sxx_template, tn_template, fn_template, ext_template = sound.spectrogram(s, fs, nperseg=1024, noverlap=512)
    Sxx_template_dB = util.power2dB(Sxx_template) + adjustment_factor
    return(Sxx_template_dB)

def create_target_spectrogram(target_path, adjustment_factor):
    s, fs = sound.load(target_path)
    Sxx, tn, fn, ext = sound.spectrogram(s, fs, nperseg=1024, noverlap=512)
    Sxx_dB = util.power2dB(Sxx) + adjustment_factor
    return(Sxx_dB, tn, fn, ext)

def calculate_corr(target_path, template_dict, adjustment_factor):
    s, fs = sound.load(target_path)
    Sxx, tn, fn, ext = sound.spectrogram(s, fs, nperseg=1024, noverlap=512)
    Sxx_dB = util.power2dB(Sxx) + adjustment_factor

    template_paths = list(template_dict.keys())
    template_prom = list(template_dict.values())
    
    for i in range(len(template_paths)):
        template = template_paths[i]
        prominence = template_prom[i]
        
        Sxx_template = create_template_spectrogram(template, adjustment_factor)
        xcorrcoef, _  = rois.template_matching(Sxx_dB, Sxx_template, 
                       tn, ext, 
                       peak_th = None,
                       display=False)

        peaks, properties = find_peaks(xcorrcoef, prominence=prominence)
        peak_times = tn[peaks]

        clip_duration = call_length[template]
        
        for t in peak_times:
            start_time = t - clip_duration / 2
            end_time = t + clip_duration / 2
            sliced = sound.trim(s, fs, min_t = start_time, max_t = end_time)
            #add saving workflow
        
    return



calculate_corr(target_path, template_dict, dB_adj)